In [19]:
import numpy as np
import pandas as pd
from scipy.sparse import linalg
import os, sys
import matplotlib.pyplot as plt

pucks = pd.read_csv('./pucks.csv')
tickets = pd.read_csv('./tickets.csv')
gates = pd.read_csv('./gates.csv')

print('Okay')

Okay


In [20]:
print('------------------PUCKS------------------')
print(pucks.info())
print('-----------------------------------------')
print(pucks.describe())
print('-----------------------------------------')

------------------PUCKS------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 12 columns):
No                 753 non-null object
ArrivingDate       753 non-null object
ArrivingTime       753 non-null object
ArrivingFlight     753 non-null object
ArrivingDI         753 non-null object
FlightType         753 non-null object
DepartDate         753 non-null object
DepartTime         753 non-null object
DepartFlight       753 non-null object
DepartDI           753 non-null object
PreviousAirport    753 non-null object
NextAirport        753 non-null object
dtypes: object(12)
memory usage: 70.7+ KB
None
-----------------------------------------
           No ArrivingDate ArrivingTime ArrivingFlight ArrivingDI FlightType  \
count     753          753          753            753        753        753   
unique    753            3          170            281          2         15   
top     PK413    21-Jan-18        20:00          *****  

In [21]:
print('------------------PUCKS------------------')
print(tickets.info())
print('-----------------------------------------')
print(tickets.describe())
print('-----------------------------------------')

------------------PUCKS------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4796 entries, 0 to 4795
Data columns (total 6 columns):
PsgID             4796 non-null object
PsgNumber         4796 non-null int64
ArrivingFlight    4796 non-null object
ArrivingDate      4796 non-null object
DepartFlight      4796 non-null object
DepartDate        4796 non-null object
dtypes: int64(1), object(5)
memory usage: 224.9+ KB
None
-----------------------------------------
         PsgNumber
count  4796.000000
mean      1.669725
std       0.470361
min       1.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       2.000000
-----------------------------------------


In [22]:
print('------------------PUCKS------------------')
print(gates.info())
print('-----------------------------------------')
print(gates.describe())
print('-----------------------------------------')

------------------PUCKS------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 6 columns):
Gate          69 non-null object
Terminal      69 non-null object
Region        69 non-null object
ArrivingDI    69 non-null object
DepartDI      69 non-null object
FlightType    69 non-null object
dtypes: object(6)
memory usage: 3.3+ KB
None
-----------------------------------------
       Gate Terminal  Region ArrivingDI DepartDI FlightType
count    69       69      69         69       69         69
unique   69        2       4          3        3          2
top      S2        S  Center          D        D          N
freq      1       41      20         39       39         45
-----------------------------------------


In [24]:
print(pucks['ArrivingDate'].values.shape)

(753,)
